In [1]:
import numpy as np
import scipy.linalg
import matplotlib.pyplot as plt

import time

# for pretty printing
np.set_printoptions(4, linewidth=100, suppress=True)

----
## Chapter 8

There are many equivalent ways of defining the determinant of a matrix, and yet more diverse methods exist for computing them.

From a computational perspective, however, some are better than others.

In [2]:
rng = np.random.default_rng(1)

n = 9
A = 2 * rng.random((n, n))

In [3]:
A

array([[1.0236, 1.9009, 0.2883, 1.8973, 0.6237, 0.8467, 1.6554, 0.8184, 1.0992],
       [0.0551, 1.507 , 1.0763, 0.6595, 1.5769, 0.6064, 0.907 , 0.2681, 0.8062],
       [0.4069, 0.5246, 1.5007, 0.5608, 0.9704, 1.9615, 1.9233, 1.4496, 1.0825],
       [0.5538, 0.3213, 1.9399, 1.0321, 0.2317, 1.247 , 1.5534, 1.226 , 1.8346],
       [0.0792, 1.0572, 0.9187, 0.1247, 1.2827, 1.7053, 1.1859, 0.5202, 1.6798],
       [1.019 , 1.0218, 1.5061, 0.2958, 1.6393, 1.3666, 1.5742, 0.3832, 1.6047],
       [0.3826, 0.1631, 1.7105, 1.7226, 1.7531, 0.9438, 0.5481, 0.0142, 1.2914],
       [1.4398, 1.6711, 0.5638, 0.4304, 1.2787, 1.6101, 1.9273, 0.301 , 0.9644],
       [1.7894, 0.8454, 1.179 , 0.049 , 1.3469, 1.8382, 1.6537, 1.771 , 1.3207]])

&nbsp;

---

&nbsp;

*The* definition of a determinant—the unique row-wise linear function satisfying certain conditions—is of course too abstract to be used in actual computations.

The so-called *Leibniz formula*
\begin{align*}
\det(A) = \sum_{\sigma : \text{permutations of $\{1, \dots, n\}$}} \operatorname{sgn}(\sigma) a_{1, \sigma(1)} \dots  a_{n, \sigma(n)}
\end{align*}
is useful when computing determinants of $2\times 2$ or $3 \times 3$ matrices. However, as there are $n!$ possible permutations of $\{1, \dots, n\}$, the Leibniz formula requires compuing the sum of $n!$ terms, being quickly intractable even for not-so-large $n$.

In [4]:
from itertools import permutations

def sign(sigma):
    # Please do not ask us why this function works...
    # If you are really curious, take MAS212 or MAS311.
    sgn = 1
    n = len(sigma)
    visited = [False] * n
    for i in range(n):
        if not visited[i]:
            cycle_len = 1
            j = i
            visited[j] = True
            while not visited[sigma[j]] :
                j = sigma[j]
                visited[j] = True
                cycle_len += 1
            if not (cycle_len & 1) :
                sgn = -sgn
    return sgn

def leibniz(A):
    det = 0
    # the function *permutations* generates
    # all the permutations of a given list.
    for sigma in permutations(range(n)):
        term = sign(sigma)
        for i in range(n):
            term *= A[i, sigma[i]]
        det += term
    return det


start = time.time()  # save current time, requires *import time*
det_leibniz = leibniz(A)
elapsed = time.time() - start

print("Computed in", elapsed, "seconds")
print()
print("det(A) =", det_leibniz)


Computed in 2.526329755783081 seconds

det(A) = -11.774961095127296


&nbsp;

---

&nbsp;

Because of the difficulty of computing the signs of permutations, the *Laplace expansion*
\begin{align*}
\det(A) = \sum_{j=1}^n (-1)^{i+j} a_{ij} \det(A_{ij})
\end{align*}
where $A_{ij}$ denotes the submatrix of $A$ obtained by removing the $i$th row and $j$th column, is more widely used when one has to compute the determinant of a matrix by hand, when the size of $A$ is usually no larger than $5\times 5$.


However, the formula is recursively defined, and while it looks like we now only have to compute the sum of $n$ terms, it is actually a sum of $n!$ terms in disguise.
Indeed,
* the determinant of a $2 \times 2$ matrix is a sum of two terms,
* and inductively, if computing the determinant of an $(n-1) \times (n-1)$ matrix requires a summation of $(n-1)!$ terms, in order to compute the determinant of an $n \times n$ matrix, we need to sum $n$ determinants of $(n-1) \times (n-1)$ matrices, hence in total we need to sum $n \times (n-1)! = n!$ terms.  

&nbsp;

So, when computing the determinant with a computer, the Laplace expansion is no better than the Leibniz formula.

In [5]:
def laplace(A):
    n = A.shape[0]
    if n == 1 :
        return A[0, 0]
    else :
        sgn = 1
        det = 0
        for j in range(n) :
            sgn = -sgn
            cols = np.concatenate([np.arange(0, j), np.arange(j+1, n)])
            det += sgn * A[0, j] * laplace(A[1:, cols])
        return det


start = time.time()
det_laplace = laplace(A)
elapsed = time.time() - start

print("Computed in", elapsed, "seconds")
print()
print("det(A) =", det_laplace)

Computed in 5.915351152420044 seconds

det(A) = -11.77496109512688


&nbsp;

---

&nbsp;

Almost all algorithms in computational linear algebra starts with decomposing the given matrix.

Computing determinants is no exception. Although we will not try to explain here why it is so, the rule of thumb is that almost all decompositions require roughly a constant multiple of $n^3$ arithmetic operations. In particular, decompositions should provide tractable methods of computing determinants, compared to the previous methods that require factorially many arithmetic operations.

We already know how to do LU decomposition, albeit in the form of
\begin{align*}
L_k P_k \dots L_1 P_1 A = U
\end{align*}
where $L_j$  are lower triangular matrices whose diagonal entries are all $1$ and $P_j$ are permutation matrices.

But this suffices for our purpose; indeed,
\begin{align*}
\det(L_k) \det(P_k) \dots \det(L_1) \det(P_1) \det(A) = \det(U),
\end{align*}
but $\det(L_j) = 1$, and $P_j$ represents a row exchange that happened in the $j$th step of the elimination, so $\det(P_j) = -1$ if there was a swap in the $j$th step of the elimination and $\det(P_j) = 1$ otherwise. Therefore,

\begin{align*}
\det(A) &= (-1)^{\text{number of row swaps}}\det(U) \\
        &= (-1)^{\text{number of row swaps}}u_{11} u_{22} \dots u_{nn}
 \end{align*} because $U$ is upper triangular.

In [6]:
def lu_det(A, eps):
    # Almost the same as the code for LU decomposition we saw
    # earlier in this semester, but with two differences:
    # (1) instead of constructing the list of P_j,
    #     only keeps track of the number of swaps occurred
    # (2) does not compute L_j, as there is no need to do so
    U = A.copy()
    n, m = U.shape
    swap_cnt = 0

    row_to_check = 0

    for j in range(m):
        if row_to_check >= n :
            break

        pivot = row_to_check
        for i in range(row_to_check+1, n):
            if abs(U[i, j]) > abs(U[pivot, j]) :
                pivot = i

        U[[row_to_check, pivot], :] = U[[pivot, row_to_check], :]

        r = row_to_check
        if r != pivot :
            swap_cnt += 1

        if abs(U[r, j]) < eps :
            pass
        else :
            for k in range(r+1, n):
                u = U[k, j] / U[r, j]
                U[k, j:] -= u * U[j, j:]
            row_to_check += 1

    det = 1
    if swap_cnt % 2 == 1 :
        det = -det

    for i in range(n):
        det = det * U[i, i]

    return det

start = time.time()
det_lu = lu_det(A, eps=1e-8)
elapsed = time.time() - start

print("Computed in", elapsed, "seconds")
print()
print("det(A) =", det_lu)

Computed in 0.0007231235504150391 seconds

det(A) = -11.774961095126896


&nbsp;

---

&nbsp;

Another popular decomposition is the QR decomposition. In computing determinants, QR decomposition is particularly interesting, because in $A = QR$ the matrix $Q$ is orthogonal hence $\det(Q) = \pm 1$.

In numpy, QR decomposition can be done with `numpy.linalg.qr`.

In fact, if one invests some time to explore the abyss of Numpy source codes, one can conclude that the orthogonal matrix $Q$ from
```
Q, R = np.linalg.qr(A)
```
is a product of $n-1$ orthogonal matrices $Q = Q_{n-1} \dots Q_2 Q_1$ where $\det(Q_j) = -1$ (unless we encounter a zero vector during the computations).



So, unless we are extremely unlucky, we can compute the determinant of $A$ as $(-1)^{n-1} r_{11} r_{22} \dots r_{nn}$.

In [7]:
def qr_det(A):
    Q, R = np.linalg.qr(A)
    det = 1
    if n % 2 == 0:
        det = -det

    for i in range(n):
        det = det * R[i, i]

    return det


start = time.time()
det_qr = qr_det(A)
elapsed = time.time() - start

print("Computed in", elapsed, "seconds")
print()
print("det(A) =", det_qr)


Computed in 0.0009446144104003906 seconds

det(A) = -11.774961095126896


In [8]:
# sanity check on det(Q)
Q, _ = np.linalg.qr(A)
print(np.linalg.det(Q))

0.9999999999999998


&nbsp;

---

&nbsp;

Of course, in real world applications, we would just use the builtin function `numpy.linalg.det`. Going with builtin functions is usually a good choice, as their performance are optimized with numerous tricks, and they come with safeguards that prevent common possible errors.

In [9]:
start = time.time()
det_np = np.linalg.det(A)
elapsed = time.time() - start

print("Computed in", elapsed, "seconds")
print()
print("det(A) =", det_np)

Computed in 0.0002582073211669922 seconds

det(A) = -11.774961095126878


&nbsp;

But how does NumPy actually compute determinants?

&nbsp;

The answer can be found in the [NumPy official document](https://numpy.org/doc/stable/reference/generated/numpy.linalg.det.html)...


> ...
>
> The determinant is computed via **LU factorization** using the LAPACK routine `z/dgetrf`.
>
> ...


So NumPy uses LU decomposition in computing determinants, while NumPy ironically does not provide an LU decomposition functionality.
